In [1]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import CSVLoader
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.agents import  AgentType, AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import  RecursiveCharacterTextSplitter
from langchain.chains import LLMChain, create_tagging_chain_pydantic
from langchain_core.tools import Tool
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field
llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo', openai_api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
loader = CSVLoader('./ToolsAndModels/Datasets/menu.csv')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
splits = text_splitter.split_documents(docs)
embeddings = GoogleGenerativeAIEmbeddings(model = 'models/embedding-001')
vector_store = Chroma.from_documents(documents=splits, embedding=embeddings,collection_name='menu_data' ,persist_directory = './Chroma/')

# 1. Create a tool
retrive_tool = create_retriever_tool(
    vector_store.as_retriever(),
    name="RecipeRetrieverTool",
    description= """### Instruction ###
        You are a good Virtual Assistant . When a user asks you about recipe/ingredients or price of a particular item you have to answer him the recipe or price of the food item based on the context and make sure
        to provide all the details.
        Context is given in triple back ticks: ```{context}```
        Question is given in double back ticks:  ``{question}``
        If the question is out of context, do not provide the wrong answer just say I am sorry i can't answer to that question.
    """
)
# 2. Prompt for agent
prompt = ChatPromptTemplate.from_messages(
    [
    (
        "system", 
        "You are an assistant that returns recipe. Please use the tool for answering the questions.Do not look up for answer elsewhere only use the tool to look for answer ."
    ),
    ("human","{question}"),
    ("placeholder","{agent_scratchpad}"),
    ("placeholder","{tool_names}"),
    ("placeholder","{tools}"),
    ]
)
# 3. Initialize an agent, AgentExecutor and run it 
tools = [retrive_tool]
agent = create_tool_calling_agent(llm = llm, tools= tools, prompt= prompt)
agent_executor = AgentExecutor(
    tools= tools, 
    agent= agent, 
    max_iterations= 2, 
    return_intermediate_steps= False, 
    verbose=True, 
    early_stopping_method ='generate')
agent_executor.invoke({"question":"what is the recipe and price of Cali Chicken Sandwich?"})



> Entering new AgentExecutor chain...

Invoking: `RecipeRetrieverTool` with `{'query': 'Cali Chicken Sandwich'}`


Item: Cali Chicken Sandwich
Description: Cali Chicken Sandwich, a fresh and flavorful option featuring grilled chicken breast topped with avocado, lettuce, tomato, and mayo.
Price: 13.95
Label: Main
Serving: 1
Recipe: Marinate chicken breasts in olive oil, lemon juice, and garlic. Grill until cooked through. Toast sandwich buns and assemble with grilled chicken, sliced avocado, lettuce, tomato, and mayonnaise. Optionally, add sprouts or bacon for extra flavor. Enjoy the bright and delicious Cali Chicken Sandwich, perfect for a light meal.

Item: Cali Chicken Sandwich
Description: Cali Chicken Sandwich, a fresh and flavorful option featuring grilled chicken breast topped with avocado, lettuce, tomato, and mayo.
Price: 13.95
Label: Main
Serving: 1
Recipe: Marinate chicken breasts in olive oil, lemon juice, and garlic. Grill until cooked through. Toast sandwich buns and ass

{'question': 'what is the recipe and price of Cali Chicken Sandwich?',
 'output': 'The recipe for Cali Chicken Sandwich is as follows:\n- Marinate chicken breasts in olive oil, lemon juice, and garlic.\n- Grill the chicken until cooked through.\n- Toast sandwich buns and assemble with grilled chicken, sliced avocado, lettuce, tomato, and mayonnaise.\n- Optionally, you can add sprouts or bacon for extra flavor.\n\nThe price of Cali Chicken Sandwich is $13.95.'}